<a href="https://colab.research.google.com/github/KaterynaRb/abtests-significance-cr/blob/main/ABtests_Significance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Update the google-cloud-bigquery library for working with
# Google BigQuery if it is already installed:
!pip install --upgrade google-cloud-bigquery

In [ ]:
# Module for working with the file system
import os

# Modules for data processing
import numpy as np
import pandas as pd
# Modules for authentication and working with BigQuery and files
from google.cloud import bigquery
from google.colab import auth, drive, files

In [ ]:
# Authentication before working with Google Cloud
auth.authenticate_user()

In [ ]:
# Create client for working with BigQuery,
# specifying ID of the working project in Google Cloud:
client = bigquery.Client(project="data-analytics-mate")

In [ ]:
# Query for retrieving data from BigQuery
query = """
WITH
  session_info AS(
  SELECT
    s.date,
    s.ga_session_id,
    sp.country,
    sp.continent,
    sp.device,
    sp.channel,
    ab.test,
    ab.test_group
  FROM
    `DA.ab_test` ab
  JOIN
    `DA.session` s
  ON
    ab.ga_session_id = s.ga_session_id
  JOIN
    `DA.session_params` sp
  ON
    ab.ga_session_id = sp.ga_session_id ),
  session_with_orders AS (
  SELECT
    session_info.date,
    session_info.country,
    session_info.continent,
    session_info.device,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    COUNT(DISTINCT o.ga_session_id) AS sessions_with_orders
  FROM
    `DA.order` o
  JOIN
    session_info
  ON
    o.ga_session_id = session_info.ga_session_id
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.continent,
    session_info.device,
    session_info.channel,
    session_info.test,
    session_info.test_group ),
  events AS (
  SELECT
    session_info.date,
    session_info.country,
    session_info.continent,
    session_info.device,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    ep.event_name,
    COUNT(ep.ga_session_id) AS event_cnt
  FROM
    `DA.event_params` ep
  JOIN
    session_info
  ON
    ep.ga_session_id = session_info.ga_session_id
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.continent,
    session_info.device,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    ep.event_name ),
  session AS (
  SELECT
    session_info.date,
    session_info.country,
    session_info.continent,
    session_info.device,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    COUNT(DISTINCT session_info.ga_session_id) AS session_cnt
  FROM
    session_info
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.continent,
    session_info.device,
    session_info.channel,
    session_info.test,
    session_info.test_group ),
  account AS (
  SELECT
    session_info.date,
    session_info.country,
    session_info.continent,
    session_info.device,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    COUNT(DISTINCT acs.ga_session_id) AS new_accounts
  FROM
    `DA.account_session` acs
  JOIN
    session_info
  ON
    acs.ga_session_id = session_info.ga_session_id
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.continent,
    session_info.device,
    session_info.channel,
    session_info.test,
    session_info.test_group )
SELECT
  session_with_orders.date,
  session_with_orders.country,
  session_with_orders.continent,
  session_with_orders.device,
  session_with_orders.channel,
  session_with_orders.test,
  session_with_orders.test_group,
  'sessions with orders' AS event_name,
  session_with_orders.sessions_with_orders AS value
FROM
  session_with_orders
UNION ALL
SELECT
  events.date,
  events.country,
  events.continent,
  events.device,
  events.channel,
  events.test,
  events.test_group,
  event_name,
  event_cnt AS value
FROM
  events
UNION ALL
SELECT
  session.date,
  session.country,
  session.continent,
  session.device,
  session.channel,
  session.test,
  session.test_group,
  'sessions' AS event_name,
  session_cnt AS value
FROM
  session
UNION ALL
SELECT
  account.date,
  account.country,
  account.continent,
  account.device,
  account.channel,
  account.test,
  account.test_group,
  'new accounts' AS event_name,
  new_accounts AS value
FROM
  account;
"""

query_job = client.query(query)  # Executing SQL-query
results = query_job.result()  # Waiting for the results

In [ ]:
# Converting results of the query to DataFrame
test_results_df = results.to_dataframe()
test_results_df.head()

,date,country,continent,device,channel,test,test_group,event_name,value
0,2020-11-03,Qatar,Asia,desktop,Organic Search,2,1,sessions with orders,1
1,2020-11-04,Kazakhstan,Asia,desktop,Undefined,2,1,sessions with orders,1
2,2020-11-04,Kuwait,Asia,mobile,Organic Search,2,2,sessions with orders,1
3,2020-11-05,Georgia,Asia,desktop,Organic Search,2,1,sessions with orders,1
4,2020-11-09,Slovakia,Europe,desktop,Organic Search,2,1,sessions with orders,1


In [ ]:
# All events
test_results_df['event_name'].value_counts()

,count
event_name,
sessions,107210
session_start,106242
page_view,101907
user_engagement,94520
first_visit,81621
scroll,73643
view_promotion,61695
view_item,44869
sessions with orders,25892


In [ ]:
# Function calculates how many times a certain event occurred,
# returns list of values (count) for chosen metric for two groups.

def get_event_values_total(
        event: str,
        test_number: int,
        datasource: pd.DataFrame = test_results_df,
        test_col: str = 'test',
        group_col: str = 'test_group',
        event_name_col: str = 'event_name',
        event_value_col: str = 'value') -> list[int]:

    # Getting data from dataframe
    df_events = (
        datasource[(datasource[test_col] == test_number) &
                    (datasource[event_name_col] == event)]
        .groupby(group_col)[event_value_col]
        .sum()
        .reset_index())

    # Sorting by group (1 than 2) and getting only column with values
    event_values = df_events.sort_values(by=group_col).iloc[:, 1].to_list()
    return event_values

In [ ]:
# Function calculates conversion rates for two groups and metric change.

def conversion_change(successes: list[int], trials: list[int]):
    conversion_a = successes[0] / trials[0]
    conversion_b = successes[1] / trials[1]
    metric_change = (conversion_b - conversion_a) / conversion_a

    return conversion_a, conversion_b, metric_change

In [ ]:
# Function calculates z-statistic and statistical significance
# of results based on 5% significance level.
from statsmodels.stats.proportion import proportions_ztest

def stat_significance(successes: list[int], trials: list[int]):
    # Perform z-test
    z_stat, p_value = np.array(proportions_ztest(successes, trials),
                               dtype=np.float32)

    if p_value < 0.05:
        is_significant = True
    else:
        is_significant = False

    return z_stat, p_value, is_significant

In [ ]:
# Function creates summary dataframe with all information
# (metrics and statistical significance):
# values of metrics, conversion rates, z-statistic and p-value.

def summary_total(
        metrics: list[str],
        num_events: list[str],
        den_events: list[str],
        datasource: pd.DataFrame = test_results_df,
        test_col: str = 'test',
        group_col: str = 'test_group',
        event_name_col: str = 'event_name',
        event_value_col: str = 'value') -> pd.DataFrame:

    # Column titles to save to a file
    column_names = ['test_number', 'metric', 'num_event', 'den_event',
         'num_count_a', 'den_count_a', 'conv_rate_a',
         'num_count_b', 'den_count_b', 'conv_rate_b',
         'metric_change', 'z_stat', 'p_value', 'significant']

    data = pd.DataFrame(columns=column_names)
    tests = sorted(datasource[test_col].unique().tolist())

    # Writing data into dataframe by rows for each test
    # and for all metrics within one test.
    for test_number in tests:
        for metric, num_event, den_event in zip(metrics, num_events, den_events):
            row = [test_number, metric, num_event, den_event]

            # Calculating numerator and denominator values
            num_event_cnt = get_event_values_total(num_event, test_number,
                                                   datasource, test_col,
                                                   group_col, event_name_col,
                                                   event_value_col)
            den_event_cnt = get_event_values_total(den_event, test_number,
                                                   datasource, test_col,
                                                   group_col, event_name_col,
                                                   event_value_col)

            # Calculating conversions for two groups and metric change
            conversion_a, conversion_b, metric_change = (
                conversion_change(num_event_cnt, den_event_cnt))

            # Writing data to row
            row.append(num_event_cnt[0])
            row.append(den_event_cnt[0])
            row.append(conversion_a)

            row.append(num_event_cnt[1])
            row.append(den_event_cnt[1])
            row.append(conversion_b)

            row.append(metric_change)

            # Perform z-test for two proportions (A and B)
            stat_signif = stat_significance(num_event_cnt, den_event_cnt)
            row.extend(stat_signif)

            # Adding row with data to result summary dataframe
            data.loc[len(data)] = row

    return(data)

In [ ]:
# Creating summary in total for 4 metrics
metrics = ['add_payment_info / sessions', 'add_shipping_info / sessions', 'begin_checkout / sessions', 'new_accounts / sessions']
num_events = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new accounts']
den_events = ['sessions', 'sessions', 'sessions', 'sessions']

df_summary_total = summary_total(metrics, num_events, den_events)
print(df_summary_total.to_markdown())

|    |   test_number | metric                       | num_event         | den_event   |   num_count_a |   den_count_a |   conv_rate_a |   num_count_b |   den_count_b |   conv_rate_b |   metric_change |    z_stat |     p_value | significant   |
|---:|--------------:|:-----------------------------|:------------------|:------------|--------------:|--------------:|--------------:|--------------:|--------------:|--------------:|----------------:|----------:|------------:|:--------------|
|  0 |             1 | add_payment_info / sessions  | add_payment_info  | sessions    |          1988 |         45362 |     0.0438252 |          2229 |         45193 |     0.0493218 |       0.12542   | -3.92488  | 8.67715e-05 | True          |
|  1 |             1 | add_shipping_info / sessions | add_shipping_info | sessions    |          3034 |         45362 |     0.0668842 |          3221 |         45193 |     0.0712721 |       0.0656048 | -2.60357  | 0.0092258   | True          |
|  2 |             1 | b

In [ ]:
# Function calculates how many times a certain event occurred,
# but broken down by category.
# Returns dataframe with values (count) for chosen metric,
# broken down by category.

def get_event_values_breakdown(
        event: str, test_number: int,
        breakdown_by: str, datasource: pd.DataFrame = test_results_df,
        test_col: str = 'test',	group_col: str = 'test_group',
        event_name_col: str = 'event_name',
        event_value_col: str = 'value') -> pd.DataFrame:

    # Getting data from dataframe into pivot table,
    # grouped by test group and given category
    df_events = (
        pd.pivot_table(datasource[(datasource[test_col] == test_number) &
                      (datasource[event_name_col] == event)],
                       values=event_value_col,
                       index=breakdown_by,
                       columns=group_col,
                       aggfunc='sum'))

    return df_events

In [ ]:
# Function creates summary dataframe with all information for data broken down
# by category (metrics and statistical significance):
# values of metrics, categories, conversion rates, z-statistic and p-value.

def summary_breakdown(
        metrics: list[str], num_events: list[str],
        den_events: list[str], breakdown_by: str,
        datasource: pd.DataFrame = test_results_df,
        test_col: str = 'test',	group_col: str = 'test_group',
        event_name_col: str = 'event_name',
        event_value_col: str = 'value') -> pd.DataFrame:

    # Column titles to save to a file
    column_names = ['test_number', breakdown_by,
          'metric', 'num_event', 'den_event',
          'num_count_a', 'den_count_a', 'conv_rate_a',
          'num_count_b', 'den_count_b', 'conv_rate_b',
          'metric_change', 'z_stat', 'p_value', 'significant']

    data = pd.DataFrame(columns=column_names)
    categories = datasource[breakdown_by].unique().tolist()
    tests = sorted(datasource[test_col].unique().tolist())

    # Writing data into dataframe by rows for each test
    # and for all metrics within one test, also broken down by category
    for test_number in tests:
        for metric, num_event, den_event in zip(metrics, num_events, den_events):
            for category in categories:
                row = [test_number, category, metric, num_event, den_event]

                # Calculating numerator and denominator values for given category
                num_event_cnt = (get_event_values_breakdown(num_event, test_number,
                                                           breakdown_by, datasource,
                                                           test_col, group_col,
                                                           event_name_col,
                                                           event_value_col).loc[category, :].to_list())

                den_event_cnt = get_event_values_breakdown(den_event, test_number,
                                                           breakdown_by, datasource,
                                                           test_col, group_col,
                                                           event_name_col,
                                                           event_value_col).loc[category, :].to_list()

                # Calculating conversions for two groups and metric change
                conversion_a, conversion_b, metric_change = (
                    conversion_change(num_event_cnt, den_event_cnt))

                # Writing data to row
                row.append(num_event_cnt[0])
                row.append(den_event_cnt[0])
                row.append(conversion_a)

                row.append(num_event_cnt[1])
                row.append(den_event_cnt[1])
                row.append(conversion_b)

                row.append(metric_change)

                # Perform z-test for two proportions (A and B)
                stat_signif = stat_significance(num_event_cnt, den_event_cnt)
                row.extend(stat_signif)

                # Adding row with data to result summary dataframe
                data.loc[len(data)] = row

    return data

In [ ]:
# Creating summary of test results for 4 metrics (by continent, device and channel)
df_summary_continent = summary_breakdown(metrics, num_events, den_events, 'continent')
df_summary_device = summary_breakdown(metrics, num_events, den_events, 'device')
df_summary_channel = summary_breakdown(metrics, num_events, den_events, 'channel')

In [ ]:
print(df_summary_continent.to_markdown())
print(df_summary_device.to_markdown())
print(df_summary_channel.to_markdown())

|    |   test_number | continent   | metric                       | num_event         | den_event   |   num_count_a |   den_count_a |   conv_rate_a |   num_count_b |   den_count_b |   conv_rate_b |   metric_change |      z_stat |     p_value | significant   |
|---:|--------------:|:------------|:-----------------------------|:------------------|:------------|--------------:|--------------:|--------------:|--------------:|--------------:|--------------:|----------------:|------------:|------------:|:--------------|
|  0 |             1 | Asia        | add_payment_info / sessions  | add_payment_info  | sessions    |           512 |         10626 |     0.0481837 |           545 |         10932 |     0.0498536 |     0.0346578   | -0.567695   | 0.570242    | False         |
|  1 |             1 | Europe      | add_payment_info / sessions  | add_payment_info  | sessions    |           324 |          8472 |     0.0382436 |           452 |          8423 |     0.0536626 |     0.403177    | -4.7

In [ ]:
# Connecting Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Changing directory
%cd /content/drive/MyDrive/MATE/PortfolioProject2

/content/drive/MyDrive/MATE/PortfolioProject2


In [ ]:
# Writing data to CSV files
df_summary_total.to_csv('summary_total.csv', index=False)

df_summary_continent.to_csv('summary_continent.csv', index=False)
df_summary_device.to_csv('summary_device.csv', index=False)
df_summary_channel.to_csv('summary_channel.csv', index=False)

In [ ]:
# Get the current working directory
cwd = os.getcwd()
print(f'The CSV files should be located in: {cwd}')

The CSV files should be located in: /content/drive/MyDrive/MATE/PortfolioProject2


[Tableau Dashboards](https://public.tableau.com/views/ABtests_17592402492710/Significance?:language=en-GB&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link)

### Files:

[Summary Total](https://drive.google.com/file/d/1yFxlnlrQ3VCpepZe5--AI33ZHLFvDJKC/view?usp=sharing)

[Summary Channel](https://drive.google.com/file/d/1POggFzWB5YjvYaAIm1VcGkezGDC869jq/view?usp=sharing)

[Summary Device](https://drive.google.com/file/d/1xzk8VVY-iUCwA773bZjD5thYxjrVT2Ie/view?usp=sharing)

[Summary Continent](https://drive.google.com/file/d/1E1luirhm1rotDXhnSe00j5i1AvospMMo/view?usp=sharing)